In [1]:
import openai
print(openai.__version__)


1.70.0


In [2]:
!pip uninstall -y openai
!pip install openai==0.28


Found existing installation: openai 1.70.0
Uninstalling openai-1.70.0:
  Successfully uninstalled openai-1.70.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
import openai


In [15]:
import getpass
openai.api_key = getpass.getpass("🔐 Enter your OpenAI API key: ")


🔐 Enter your OpenAI API key: ··········


In [3]:
df = pd.read_csv("/sample.csv")  # Replace with your exact file name if different
df.head()


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [4]:
def classify_message(text):
    prompt = f"""Classify this support message into one of the following categories:
    [Login Issue, Billing, Technical, Complaint, Feedback, Other]

    Message: "{text}"
    Category:"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("❌ Error:", e)
        return "Error"


In [5]:
print("🔁 Starting test tagging...")

df_sample = df.head(5).copy()  # Try with first 5 rows
df_sample["category"] = df_sample["text"].apply(classify_message)

print("✅ Tagging complete!")
df_sample[["text", "category"]]


🔁 Starting test tagging...
❌ Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
❌ Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
❌ Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
❌ Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
❌ Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs

,text,category
0,@AppleSupport causing the reply to be disregar...,Error
1,@105835 Your business means a lot to us. Pleas...,Error
2,@76328 I really hope you all change but I'm su...,Error
3,@105836 LiveChat is online at the moment - htt...,Error
4,@VirginTrains see attached error message. I've...,Error


In [7]:
import pandas as pd

# Load your uploaded file (replace with the actual name)
df = pd.read_csv("/sample.csv")

# Show the first few rows
df.head()


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [8]:
def keyword_categorize(text):
    text = str(text).lower()  # make lowercase for matching

    if "login" in text or "log in" in text:
        return "Login Issue"
    elif "charge" in text or "billing" in text or "payment" in text:
        return "Billing"
    elif "order" in text or "shipping" in text or "deliver" in text:
        return "Shipping"
    elif "crash" in text or "bug" in text or "error" in text:
        return "Technical"
    elif "change email" in text or "update info" in text:
        return "Account Update"
    elif "thank" in text:
        return "Positive Feedback"
    elif "sucks" in text or "worst" in text or "angry" in text:
        return "Complaint"
    else:
        return "Other"


In [9]:
df["category"] = df["text"].apply(keyword_categorize)
df[["text", "category"]].head()


,text,category
0,@AppleSupport causing the reply to be disregar...,Other
1,@105835 Your business means a lot to us. Pleas...,Other
2,@76328 I really hope you all change but I'm su...,Other
3,@105836 LiveChat is online at the moment - htt...,Other
4,@VirginTrains see attached error message. I've...,Technical


In [12]:
df.to_csv("/sample_twitter.csv", index=False)


In [13]:
from google.colab import files
files.download("/sample_twitter.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>